In [1]:
import os

from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'


from operator import sub
import json
import os
import cv2
import json
import glob
import tqdm
import torch
import shutil
import subprocess
import numpy as np
import esim_torch
from upsampling import *
from esim_torch import esim_torch
from upsampling.utils import Upsampler
import esim_torch as et
#from general_event_generator.events_generator import EventGenerator

In [2]:
import os
import json
import shutil
import subprocess



class VideoProcessor:

    def __get_video_info(self):

            try:

                """Calculates the proberities of a video file using ffprobe. """

                properties_cmd = ["ffprobe", "-v", "error", "-select_streams", "v:0", "-show_entries",
                        "stream=height,width,nb_frames,duration", "-of", "json", 
                        self.video_path]
                
                result = subprocess.run(properties_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                                        text=True)

                # Parse the JSON output to get the duration.
                
                output_json = json.loads(result.stdout)
                
                width = int(output_json["streams"][0]["width"])
                height = int(output_json["streams"][0]["height"])
                duration = float(output_json["streams"][0]["duration"])
                frame_count = int(output_json["streams"][0]["nb_frames"])
                fps = int(frame_count/duration)
                return width, height, duration, frame_count, fps
            
            except:
                print("No VIDEO output found")
                return None

    def __init__(self, video_path, cut_part_duration = 2, resize_scale=0.3, debug=False):
        """ 
        Class that preprocess video, devide it into chuncks and 
        """
        self.video_path = video_path
        self.cut_part_duration = cut_part_duration
        self.resize_scale = resize_scale
        self.debug = debug

        self.video_name = os.path.basename(self.video_path).split(".")[0]
        
        self.orignal_folder_path = f"{os.path.dirname(self.video_path)}/orginal/"
        self.upsampling_dir =f"{os.path.dirname(self.video_path)}/upsampled/"
        self.events_dir = f"{os.path.dirname(self.video_path)}/events/"

        if not os.path.exists(self.orignal_folder_path):
                os.mkdir( self.orignal_folder_path)

        if debug:
            print(f"video_name: {self.video_name}")
            print(f"orignal_folder_path: {self.orignal_folder_path}")
        
        try:
            self.width, self.height, self.video_duration, self.total_frames, \
                                        self.fps = self.__get_video_info()
            self.num_cut_parts = int(self.video_duration) // int(self.cut_part_duration)
            

        except Exception as e:
            print(f"Error getting video info: {e}")
            print("---")

            self.width, self.height, self.video_duration, self.total_frames, \
                self.fps = None, None, None, None, None
            print(self.width, self.height, self.video_duration, self.total_frames, \
                self.fps) 
   
    def cut_resize_video(self):
        """
            Use this function as the first step to cut and resize the video.

        """
        for i in range(self.num_cut_parts):
            if i == 0:
                cut_part_start_time = 0
            else:
                cut_part_start_time = i + self.cut_part_duration
                
            cut_part_end_time = cut_part_start_time + self.cut_part_duration

            # Handle last video duration
            if (cut_part_start_time + self.cut_part_duration) > self.video_duration :
                cut_part_end_time = self.video_duration
                
                
            #create a cut sequence folder
            
            video_parts_path = os.path.join(self.orignal_folder_path, f"seq_{i}")


            if not os.path.exists(video_parts_path):
                os.mkdir( video_parts_path)

            # Create the output file path for the cut part
            cut_part_output_path = os.path.join(video_parts_path, f"{self.video_name}_part_{i}.mp4")

            if self.debug:

                print(f"cut_part_start_time: {cut_part_start_time}", cut_part_start_time, end=" - ")
                print(f"cut_part_end_time: {cut_part_end_time}", end=" - ")
                print(f"video_parts_path: {video_parts_path}", end=" - ")
                print(f"cut_part_output_path: {cut_part_output_path}", end=" - ")
                print("----------------------------------------------")
                

            # Cut and resize the video
            cut_resize_cmd = ["ffmpeg", "-i", self.video_path, 
                            "-ss", str(cut_part_start_time), 
                            "-t", str(cut_part_end_time),
                            "-vf", f"scale={self.resize_scale}*iw:{self.resize_scale}*ih",
                            cut_part_output_path]
            subprocess.run(cut_resize_cmd)
            

            # Check if the video if empty through number of frames
            print(self.video_duration)
            if self.__get_video_info() is None:

                print(cut_part_output_path)
                print(" video cut error")
                shutil.rmtree(video_parts_path)
                print(f"cut_part_end_time: {cut_part_end_time} ")
                print(f"The whole video duration is {self.duration}")

                break
            else:
                # Add a subprocess to extract video frames from the new video chuncks

                cut_part_frames_path = os.path.join(video_parts_path, "imgs")
                
                if not os.path.exists(cut_part_frames_path):
                    os.mkdir( cut_part_frames_path)

                vid2frame_cmd = ["ffmpeg",
                        "-i",
                        cut_part_output_path,
                        f"{cut_part_frames_path}/%08d.png",]

                subprocess.run(vid2frame_cmd)

                # Create fps.txt file
                fps_txt_path = os.path.join(video_parts_path, "fps.txt")
                
                with open(fps_txt_path, "w") as f:
                    f.write(str(self.fps))


    def upsample_video(self):
            """
                Use this function as the 2nd step to upsample video chuncks using interpolation between frames.
            """
            
            try:
                upsampler = Upsampler(input_dir=self.orignal_folder_path, output_dir=self.upsampling_dir)
                
                upsampler.upsample()
            except Exception as e:
                print(f"{e}, please check upsampling code!")


    def generate_timestamps(self, frame_rate):
        """
            use those functions at the last to generate timestamps for the upsamppled frames. 
        """
        for folder in folders:
            images = sorted(
                [f for f in listdir( args.input_folder+folder+"/imgs") if f.endswith('.png')])
            time_path = args.input_folder + folder 

            print('Will write file: {} with framerate: {} Hz'.format(\
                time_path + '/timestamps.txt',  args.framerate))
            
            stamp_nanoseconds = 1
            dt_nanoseconds = int((1.0 /  args.framerate) * 1e9 )


            with open(join(time_path, 'timestamps.txt'), 'w') as f:
                for image_path in images:

                    f.write('{}\n'.format(int(stamp_nanoseconds)))
                    stamp_nanoseconds += dt_nanoseconds
                f.close()
        


    def __is_valid_dir(subdirs, files):
        return len(subdirs) == 1 and len(files) == 1 and "timestamps.txt" in files and "imgs" in subdirs


    def __process_dir(outdir, indir, contrast_threshold_negative, contrast_threshold_positive, refractory_period_ns):
        print(f"Processing folder {indir}... Generating events in {outdir}")
        os.makedirs(outdir, exist_ok=True)

        # constructor
        esim = et.ESIM(contrast_threshold_negative,
                            contrast_threshold_positive,
                            refractory_period_ns)

        timestamps = np.genfromtxt(os.path.join(indir, "timestamps.txt"), dtype="float64")
        timestamps_ns = (timestamps * 1e9).astype("int64")
        timestamps_ns = torch.from_numpy(timestamps_ns).cuda()

        image_files = sorted(glob.glob(os.path.join(indir, "imgs", "*.png")))
        
        pbar = tqdm.tqdm(total=len(image_files)-1)
        num_events = 0

        counter = 0
        for image_file, timestamp_ns in zip(image_files, timestamps_ns):
            image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
            log_image = np.log(image.astype("float32") / 255 + 1e-5)
            log_image = torch.from_numpy(log_image).cuda()

            sub_events = esim.forward(log_image, timestamp_ns)

            # for the first image, no events are generated, so this needs to be skipped
            if sub_events is None:
                continue

            sub_events = {k: v.cpu() for k, v in sub_events.items()}    
            num_events += len(sub_events['t'])


    def generate_events(self, contrast_threshold_negative, contrast_threshold_positive, refractory_period_ns):
         
        input_dir = self.upsampling_dir
        output_dir = self.events_dir

        if not os.path.exists(input_dir):
            raise   Exception(f"{Exception}: Please use 1st and 2nd steps firist to generate events!")
        
        if not os.path.exists(output_dir):
            os.mkdir(output_dir)

        print(f"Generating events with cn={contrast_threshold_negative}, cp={contrast_threshold_positive} \
              and rp={refractory_period_ns}")

        for path, subdirs, files in os.walk(input_dir):
            if self.__is_valid_dir(subdirs, files):
                output_folder = os.path.join(output_dir, os.path.relpath(path, input_dir))

                self.__process_dir(output_folder, path)
        
                
"""
    Developed by: Eman Ehab Nasef
"""

'\n    Developed by: Eman Ehab Nasef\n'

In [3]:
v_pth = "./Archive-code/Test_vidCutResizer/video_0003.mp4"
vcr = VideoProcessor(v_pth, cut_part_duration=7, debug=True)
vcr.upsample_video()

video_name: video_0003
orignal_folder_path: ./Archive-code/Test_vidCutResizer/orginal/


2023-11-04 11:43:20.528786: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-04 11:43:20.530779: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-04 11:43:20.531540: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-04 11:43:20.533295: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Processing sequence number ./Archive-code/Test_vidCutResizer/orginal/seq_5


ImageSequence:   0%|          | 0/749 [00:00<?, ?it/s]2023-11-04 11:43:26.071511: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2023-11-04 11:43:29.432484: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8500
ImageSequence:   2%|▏         | 13/749 [00:41<34:30,  2.81s/it] 

In [ ]:
framerate = 1200
v_pth = "./Archive-code/Test_vidCutResizer/video_0003.mp4"
contrast_threshold_negative = 0.2
contrast_threshold_positive = 0.2
refractory_period_ns = 0


vcr = VideoProcessor(v_pth, cut_part_duration=10, debug=True)
vcr.generate_events(contrast_threshold_negative, contrast_threshold_positive, refractory_period_ns)

video_name: video_0003
orignal_folder_path: ./Archive-code/Test_vidCutResizer/orginal/
Generating events with cn=0.2, cp=0.2               and rp=0


TypeError: __is_valid_dir() takes 2 positional arguments but 3 were given

In [ ]:
framerate = 1200
v_pth = "./Archive-code/Test_vidCutResizer/video_0003.mp4"
contrast_threshold_negative = 0.2
contrast_threshold_positive = 0.2
refractory_period_ns = 0


input_dir = f"{os.path.dirname(self.video_path)}/upsampled/"
output_dir = f"{os.path.dirname(self.video_path)}/events/"



In [ ]:
v_pth = "./Archive-code/Test_vidCutResizer/video_0003.mp4"
vcr = VideoProcessor(v_pth, cut_part_duration=10, debug=True)
#vcr.cut_resize_video()

video_name: video_0003
orignal_folder_path: ./Archive-code/Test_vidCutResizer/orginal/


In [ ]:
sampling_input_dir = f"{os.path.dirname(v_pth)}/orginal/"
sampling_output_dir =f"{os.path.dirname(v_pth)}/upsampled/"

upsampler = Upsampler(input_dir=sampling_input_dir, output_dir=sampling_output_dir)

upsampler.upsample()

Processing sequence number ./Archive-code/Test_vidCutResizer/orginal/seq_5


ImageSequence:   4%|▍         | 17/449 [00:33<14:12,  1.97s/it]


KeyboardInterrupt: 

In [ ]:

class TimestampsUpsampler:

    def __init__(self, upsample_dir, framerate):
        self.upsample_dir = upsample_dir
        self.framerate = framerate

    def generate_timestamps(self):

        for i, seq_path in enumerate(self.upsample_dir):

            images = sorted(
                [f for f in os.listdir(seq_path +"/imgs") if f.endswith('.png')])

            print('Will write file: {} with framerate: {} Hz'.format(\
                seq_path + '/timestamps.txt',  self.framerate))
            
            stamp_nanoseconds = 1
            dt_nanoseconds = int((1.0 /  self.framerate) * 1e9 )


            with open(os.path.join(seq_path, 'timestamps.txt'), 'w') as f:
                for image_path in images:

                    f.write('{}\n'.format(int(stamp_nanoseconds)))
                    stamp_nanoseconds += dt_nanoseconds
                f.close()


In [ ]:
class EventGenerator:
    def __init__(self, contrast_threshold_negative, contrast_threshold_positive, refractory_period_ns):
        self.contrast_threshold_negative = contrast_threshold_negative
        self.contrast_threshold_positive = contrast_threshold_positive
        self.refractory_period_ns = refractory_period_ns
        self.esim = esim_torch.ESIM(contrast_threshold_negative,
                                    contrast_threshold_positive,
                                    refractory_period_ns)
    
    @staticmethod
    def is_valid_dir(subdirs, files):
        return len(subdirs) == 1 and len(files) == 1 and "timestamps.txt" in files and "imgs" in subdirs

    @staticmethod
    def is_valid_dir(subdirs, files):
        return len(subdirs) == 1 and len(files) == 1 and "timestamps.txt" in files and "imgs" in subdirs

    def process_dir(self, outdir, indir):
        print(f"Processing folder {indir}... Generating events in {outdir}")
        os.makedirs(outdir, exist_ok=True)

        timestamps = np.genfromtxt(os.path.join(indir, "timestamps.txt"), dtype="float64")
        timestamps_ns = (timestamps * 1e9).astype("int64")
        timestamps_ns = torch.from_numpy(timestamps_ns).cuda()

        image_files = sorted(glob.glob(os.path.join(indir, "imgs", "*.png")))
        
        pbar = tqdm.tqdm(total=len(image_files)-1)
        num_events = 0

        counter = 0
        for image_file, timestamp_ns in zip(image_files, timestamps_ns):
            image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
            log_image = np.log(image.astype("float32") / 255 + 1e-5)
            log_image = torch.from_numpy(log_image).cuda()

            sub_events = self.esim.forward(log_image, timestamp_ns)

            if sub_events is None:
                continue

            sub_events = {k: v.cpu() for k, v in sub_events.items()}    
            num_events += len(sub_events['t'])
     
            np.savez(os.path.join(outdir, "%010d.npz" % counter), **sub_events)
            pbar.set_description(f"Num events generated: {num_events}")
            pbar.update(1)
            counter += 1

        


In [ ]:
class EventGenerationManager:
    
    def __init__(self, video_path, contrast_threshold_negative, contrast_threshold_positive, refractory_period_ns):
        
        self.input_dir = f"{os.path.dirname(video_path)}/upsampled/"
        self.output_dir = f"{os.path.dirname(video_path)}/events/"
        if not os.path.exists(self.output_dir):
            os.mkdir( self.output_dir)
        
        self.contrast_threshold_negative = contrast_threshold_negative
        self.contrast_threshold_positive = contrast_threshold_positive
        self.refractory_period_ns = refractory_period_ns
    
    def process_directories(self, event_generator):
        for path, subdirs, files in os.walk(self.input_dir):
            if EventGenerator.is_valid_dir(subdirs, files):
                output_folder = os.path.join(self.output_dir, os.path.relpath(path, self.input_dir))
                event_generator.process_dir(output_folder, path)

    def start_event_generation(self):
        event_gen = EventGenerator(self.contrast_threshold_negative, self.contrast_threshold_positive, self.refractory_period_ns)
        print(f"Generating events with cn={event_gen.contrast_threshold_negative}, cp={event_gen.contrast_threshold_positive} and rp={event_gen.refractory_period_ns}")
        self.process_directories(event_gen)




In [ ]:
# Define parameters directly in the script
v_pth = "./Archive-code/Test_vidCutResizer/video_0003.mp4"
contrast_threshold_negative = 0.2
contrast_threshold_positive = 0.2
refractory_period_ns = 0

manager = EventGenerationManager(v_pth, contrast_threshold_negative, contrast_threshold_positive, refractory_period_ns)
manager.start_event_generation()

Generating events with cn=0.2, cp=0.2 and rp=0
Processing folder ./example2/upsampled/dirname_does_not_matter... Generating events in ./example2/events/dirname_does_not_matter


Num events generated: 806137: 100%|██████████| 64/64 [00:00<00:00, 140.48it/s]


Processing folder ./example2/upsampled/seq2... Generating events in ./example2/events/seq2


Num events generated: 69442: : 17it [00:00, 175.47it/s]                      


Processing folder ./example2/upsampled/seq0... Generating events in ./example2/events/seq0


Num events generated: 300538: : 65it [00:00, 159.52it/s]                      


Processing folder ./example2/upsampled/seq1... Generating events in ./example2/events/seq1


Num events generated: 12337: : 65it [00:00, 180.54it/s]                      


In [ ]:
event_ns = np.load("./example2/events/seq0/0000000049.npz")

In [ ]:
for key in event_ns.keys():
    print(key)

x
y
t
p


In [ ]:
    
import os
import numpy as np
import cv2

from skimage import io

path = './example2/events/seq0/'

npz_files = sorted(os.listdir(path))

for file in npz_files:
    frame_data = np.load(path+str(file))

    # Extract x, y, t, and p arrays from the npz file
    x = frame_data['x']
    y = frame_data['y']
    t = frame_data['t']
    p = frame_data['p']

    # Convert event-based data to frame
    try:
        max_x = np.max(x)
        max_y = np.max(y)
        frame = np.zeros((max_y + 1, max_x + 1), dtype=np.uint8)

        # Iterate through the events and update the frame
        for i in range(len(x)):
            frame[y[i], x[i]] = 255 if p[i] else 0
            cv2.imshow('Frame', frame)
            cv2.waitKey(0.9)
    except:
        pass

        
cv2.destroyAllWindows()

# Example usage
